In [397]:

# Imports
import sys
import pandas as pd
import numpy as np
np.set_printoptions(threshold=sys.maxsize)



In [398]:
# PROCEDURE

# 1). Hot Encode Difficulty Level column
# 2). Train Test split data
# 3). TF-IDF on Train
# 4). TF on Test

# Notes For Possible Optimization

# 1). In TF-IDF we can update stop words such that they are custom to LeetCode problems. 

# THESE ARE THE HYPER_PARAMETERS
n_gram_range_text = (1,3)
n_gram_range_title = (1,3)
max_feature_tfidf_question_text = 1500
max_feature_tfidf_question_title = 1500

df_original = pd.read_csv('trimed_leetcode_questions.csv');
df_original = df_original.drop(columns=['Topics'])
df_original.head(3)


,Question Title,Question Text,Difficulty Level,Array,String,Dynamic Programming,Math,Hash Table,Greedy,Sorting,Depth-First Search,Breadth-First Search,Binary Search
0,Two Sum,Given an array of integers nums and an integer...,Easy,1,0,0,0,1,0,0,0,0,0
1,Add Two Numbers,You are given two non-empty linked lists repre...,Medium,0,0,0,1,0,0,0,0,0,0
2,Longest Substring Without Repeating Characters,"Given a string s, find the length of the longe...",Medium,0,1,0,0,1,0,0,0,0,0


In [399]:
#   STEP 1

# One Hot Encoding on Difficuly Level 
# Drop the first column in basic one hot encoding (Only Medium and Hard is needed to determine if question is Easy, Medium or Hard)
# Drop Original column that was Hot Encoded
from sklearn.preprocessing import OneHotEncoder

oenc=OneHotEncoder(drop='first');
difficulty_enc=pd.DataFrame(oenc.fit_transform(df_original[['Difficulty Level']]).toarray());
df_difficulty_enc=pd.concat([df_original,difficulty_enc],axis=1)
df_difficulty_enc.rename(columns={0:'Hard', 1:'Medium'},inplace=True)
df_difficulty_enc = df_difficulty_enc[['Question Title', 'Question Text', 'Difficulty Level','Hard', 'Medium','Array', 'String', 'Dynamic Programming', 'Math', 'Hash Table','Greedy', 'Sorting', 'Depth-First Search', 'Breadth-First Search','Binary Search']]
df = df_difficulty_enc.drop(columns=['Difficulty Level'])
df.head(3)




,Question Title,Question Text,Hard,Medium,Array,String,Dynamic Programming,Math,Hash Table,Greedy,Sorting,Depth-First Search,Breadth-First Search,Binary Search
0,Two Sum,Given an array of integers nums and an integer...,0.0,0.0,1,0,0,0,1,0,0,0,0,0
1,Add Two Numbers,You are given two non-empty linked lists repre...,0.0,1.0,0,0,0,1,0,0,0,0,0,0
2,Longest Substring Without Repeating Characters,"Given a string s, find the length of the longe...",0.0,1.0,0,1,0,0,1,0,0,0,0,0


In [400]:
#   STEP 2

#Train Test Split 80 - 20 (Train - Test)
from sklearn.model_selection import train_test_split

X = df.drop(columns = ['Array', 'String', 'Dynamic Programming', 'Math', 'Hash Table','Greedy', 'Sorting', 'Depth-First Search', 'Breadth-First Search','Binary Search'])
y = df.drop(columns=['Question Title', 'Question Text','Hard', 'Medium'])
X_train_pretfidf, X_test_pretfidf, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123, shuffle=True)
X_train_pretfidf.head(3)


,Question Title,Question Text,Hard,Medium
1106,Probability of a Two Boxes Having The Same Num...,Given 2n balls of k distinct colors. You will ...,1.0,0.0
116,Surrounded Regions,Given an m x n matrix board containing 'X' and...,0.0,1.0
1001,Delete Leaves With a Given Value,Given a binary tree root and an integer target...,0.0,1.0


In [401]:
#   STEP 3
# Preform Tfidf on Train
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_text = TfidfVectorizer(analyzer='word', max_features=max_feature_tfidf_question_text, ngram_range=n_gram_range_text, stop_words='english')
df_train_description_matrix = tfidf_text.fit_transform(X_train_pretfidf['Question Text'])
df_train_description = pd.DataFrame(df_train_description_matrix.toarray(), columns = tfidf_text.get_feature_names_out())


tfidf_title = TfidfVectorizer(analyzer='word', max_features=max_feature_tfidf_question_title, ngram_range=n_gram_range_title, stop_words='english')
df_train_title_matrix = tfidf_title .fit_transform(X_train_pretfidf['Question Title'])
df_train_title = pd.DataFrame(df_train_title_matrix.toarray(), columns = tfidf_title.get_feature_names_out())
X_train_pretfidf_reset = X_train_pretfidf.reset_index();



X_train = pd.concat([X_train_pretfidf_reset, df_train_title, df_train_description], axis=1)
X_train = X_train.drop(columns=['index', 'Question Title', 'Question Text'])
X_train.head(3)



,Hard,Medium,2d,2d grid,2d matrix,3d,3d shapes,3sum,4sum,absolute,...,xi yi,xor,xor elements,y1,y1 x2,y1 x2 y2,y2,yi,zero,zeros
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [402]:
#   STEP 4
# Preform Tf on Test


df_test_description_matrix = tfidf_text.transform(X_test_pretfidf['Question Text'])
df_test_description = pd.DataFrame(df_test_description_matrix.toarray(), columns = tfidf_text.get_feature_names_out())


df_test_title_matrix = tfidf_title .transform(X_test_pretfidf['Question Title'])
df_test_title = pd.DataFrame(df_test_title_matrix.toarray(), columns = tfidf_title.get_feature_names_out())
X_test_pretfidf_reset = X_test_pretfidf.reset_index();



X_test = pd.concat([X_test_pretfidf_reset, df_test_title, df_test_description], axis=1)
X_test = X_test.drop(columns=['index','Question Title', 'Question Text'])
X_test.head(3)




,Hard,Medium,2d,2d grid,2d matrix,3d,3d shapes,3sum,4sum,absolute,...,xi yi,xor,xor elements,y1,y1 x2,y1 x2 y2,y2,yi,zero,zeros
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
